In [18]:
import sys

import json
from nornir import InitNornir
from nornir.core.filter import F

from nornir.core.plugins.inventory import InventoryPluginRegister
from nornir_ansible.plugins.inventory.ansible import AnsibleInventory

from nornir_utils.plugins.functions import print_result
#from nornir_napalm.plugins.tasks import napalm_get

from nornir_netmiko import netmiko_send_command
#from netmiko import ssh_exception

InventoryPluginRegister.register("inventory", AnsibleInventory)

In [19]:
def nornir_connect_and_run_command(task, plugin, action, params, platform, username, password):
  task.host.open_connection(plugin, configuration=task.nornir.config, platform=platform, username=username, password=password)
  r = task.run(
    task=action,
    command_string=params[0]
  )
  task.host.close_connection(plugin)

In [20]:
topology="2host"

In [21]:
# Initialize Nornir object with Containerlab ansible inventory
nrinit = InitNornir(
  runner={
      "plugin": "threaded",
      "options": {
          "num_workers": 10,
      },
  },
  inventory={
      "plugin": "AnsibleInventory",
      "options": {
          "hostsfile": f"../clab/clab-{topology}/ansible-inventory.yml"
      },
  },
)

kinds_platforms = {
#    'ceos':     'eos',
#    'crpd':     'junos', 
    'linux':     'linux', 
#    'vr-veos':  'eos',
#    'vr-vmx':   'junos', 
#    'vr-xrv9k': 'iosxr',
}

kinds_credentials = {
    'linux':    {"username": "root",  "password": "root"},
    'ceos':     {"username": "admin", "password": "admin"},
    'crpd':     {"username": "root",  "password": "clab123"},
}

kinds_tasks = {
    'linux':     nornir_connect_and_run_command,
    #'ceos':      nornir_connect_and_run_getters,
    'crpd':      nornir_connect_and_run_command,
}

kinds_plugins = {
    'linux':     "netmiko",
    'ceos':      "napalm",
    'crpd':      "netmiko",
}

kinds_actions = {
    'linux':     netmiko_send_command,
    #'ceos':      napalm_get,
    'crpd':      netmiko_send_command,
}

kinds_params = {
    'linux':    ["ip -json address show"], # single element only
    #'ceos':     ["facts", "interfaces", "lldp_neighbors"],
    'crpd':     ["ip -json address show"], # single element only
}

node_data = {
    "name": topology,
    "type": "node-data",
    "nodes": {},
}

nodes = {}
results = []
for k, v in kinds_platforms.items():
    nr = nrinit.filter(F(groups__contains=k))
    r = nr.run(
      task=kinds_tasks[k],
      plugin=kinds_plugins[k],
      action=kinds_actions[k],
      params=kinds_params[k],
      platform=v,
      username=kinds_credentials[k]["username"],
      password=kinds_credentials[k]["password"],
    )
    results.append(r)

In [22]:
for r in results:
    for k, v in r.items():
        if not v[0].failed:
            j = json.loads(v[1].result)
            nodes |= {k: {"model": "linux", "interfaces": j}}
        else:
            print(f"Connection failed for: {k}. Error: {v[0]}")

In [23]:
node_data["nodes"] |= nodes
print(json.dumps(node_data, indent=4))

{
    "name": "2host",
    "type": "node-data",
    "nodes": {
        "clab-2host-host1": {
            "model": "linux",
            "interfaces": [
                {
                    "ifindex": 1,
                    "ifname": "lo",
                    "flags": [
                        "LOOPBACK",
                        "UP",
                        "LOWER_UP"
                    ],
                    "mtu": 65536,
                    "qdisc": "noqueue",
                    "operstate": "UNKNOWN",
                    "group": "default",
                    "txqlen": 1000,
                    "link_type": "loopback",
                    "address": "00:00:00:00:00:00",
                    "broadcast": "00:00:00:00:00:00",
                    "addr_info": [
                        {
                            "family": "inet",
                            "local": "127.0.0.1",
                            "prefixlen": 8,
                            "scope": "host",
               

}
